## Carga de librerías necesarias

In [2]:
from IPython.display import Javascript, display
from ipywidgets import widgets

def run_all(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))

button = widgets.Button(description="Press here to execute everything")
button.on_click(run_all)
display(button)

Button(description='Press here to execute everything', style=ButtonStyle())

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import plotly.express as px  #https://plotly.com/python/line-and-scatter/
from ipywidgets import fixed, interact_manual
from ipywidgets import interact
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings('ignore')
import webbrowser

## Carga de datos y preprocesado

In [4]:
#df = pd.read_excel('owid-energy-data.xlsx')
df = pd.read_excel('https://nyc3.digitaloceanspaces.com/owid-public/data/energy/owid-energy-data.xlsx')

df["gdp per capita"]=df.gdp/df.population #creo nuevo atributo PIB per capita
df["gdp per capita"]=df["gdp per capita"].round(2)


countries=list(set(df.country))
countries_none=countries.copy()
countries_none.append('None')
atributos=list(df.columns)
atributos_num=[atributo for atributo in atributos if df[atributo].dtypes=="float64"]

#df_metadatos = pd.read_excel('owid-energy-data.xlsx',sheet_name=1)  
df_metadatos = pd.read_excel('https://nyc3.digitaloceanspaces.com/owid-public/data/energy/owid-energy-data.xlsx',sheet_name=1)
df_metadatos

dict_unidades=dict()

for columna in atributos:
    if columna != 'gdp per capita':
        try: dict_unidades[columna]=df_metadatos[df_metadatos.column==columna].unit.values[0]
        except: dict_unidades[columna]=df_metadatos[df_metadatos.column==columna].unit.values
    else: dict_unidades[columna]='$/person'
dict_unidades




# cargo df con poligonos del mapa mundial
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))

world
#En primer lugar asignamos el iso 'KOS' a 'Kosovo' en ambos dataframes:
world.iso_a3[world.name=='Kosovo']='KOS'
df.iso_code[df.country=='Kosovo']='KOS'



###Añado la columna continente al df a partir de la información en el dataframe world

dict_countries=dict();
for i in range(world.shape[0]):
    dict_countries[world.iloc[i].iso_a3]=world.iloc[i].continent

continent_series=pd.Series(data=list("c"*df.shape[0]))

for i in range(continent_series.shape[0]):
    try:
        continent_series.iloc[i]=dict_countries[df.iloc[i].iso_code]
    except:
        continent_series.iloc[i]=None

continent_series

df["continent"]=continent_series







## Visualización 1

In [4]:
!pip install Dash

In [5]:
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go

app1 = Dash(__name__)

app1.layout = html.Div([
    html.H1(children='Diagrama de dispersión',style={'textAlign':'center'}),
    html.P("Select year and atributes:"),
    dcc.Dropdown(id="dropdown11",options=atributos_num, value='population', clearable=False,),
    dcc.Dropdown(id="dropdown12",options=countries,value='Spain',clearable=False,),
    dcc.Dropdown(id="dropdown13",options=countries_none,value='None',clearable=False,),
    dcc.Dropdown(id="dropdown14",options=countries_none,value='None',clearable=False,),
    dcc.Dropdown(id="dropdown15",options=countries_none,value='None',clearable=False,),
    dcc.Graph(id="graph1")], style={ 'height':'5000'})


@app1.callback(
    Output("graph1", "figure"),
    Input("dropdown11", "value"),Input("dropdown12", "value"),Input("dropdown13", "value"),Input("dropdown14", "value"),
    Input("dropdown15", "value"))

def temp_series(atributo="population",country1="Spain", country2="None",country3="None",country4="None"):
    df_countries=df[(df.country==country1)|(df.country==country2)|(df.country==country3)|(df.country==country4)]
    fig1 = px.line(df_countries,x="year",y=atributo,color='country',
                  labels={
                     "year": "year",
                     atributo: f'{atributo} ({dict_unidades[atributo]})'},
                  title=f'Temporal series of {atributo}')
    fig1.update_layout(plot_bgcolor="white")
    fig1.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
    fig1.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
    return fig1


if __name__ == '__main__':
    webbrowser.open("http://127.0.0.1:7080/")
    app1.run(host='127.0.0.1', port='7080', proxy=None, debug=False,
             dev_tools_ui=None, dev_tools_props_check=None, dev_tools_serve_dev_bundles=None,
             dev_tools_hot_reload=None, dev_tools_hot_reload_interval=None,
             dev_tools_hot_reload_watch_interval=None,
             dev_tools_hot_reload_max_retry=None, dev_tools_silence_routes_logging=None,
             dev_tools_prune_errors=None)





<html><head>


<!-- Load require.js. Delete this if your page already loads require.js -->
<script src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.4/require.min.js" integrity="sha256-Ae2Vz/4ePdIu6ZyI/5ZGsYnb+m0JlOmKPjt6XZ9JJkA=" crossorigin="anonymous"></script>
<script src="https://cdn.jsdelivr.net/npm/@jupyter-widgets/html-manager@*/dist/embed-amd.js" crossorigin="anonymous"></script>
<script type="application/vnd.jupyter.widget-state+json">
{
    "version_major": 2,
    "version_minor": 0,
    "state": {
        "bc697fe8975b43dabd75bfb60389b7b4": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "2.0.0",
            "state": {
                "value": 1900,
                "max": 2022,
                "min": 1900,
                "style": "IPY_MODEL_d80cdfa24b4a4185822d8307dde99a33",
                "behavior": "drag-tap",
                "layout": "IPY_MODEL_d3368f1cb4dd48ce815ff85d087dfaa8"
            }
        }
    }
}
</script>
</head>
<body>

<script type="application/vnd.jupyter.widget-view+json">
{
    "version_major": 2,
    "version_minor": 0,
    "model_id": "add65cfd37e2451387ff496bb6293e56"
}
</script>

</body>
</html>


## Visualización 2

In [6]:
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go

app2 = Dash(__name__)

app2.layout = html.Div([
    html.H1('Diagrama de dispersión',style={'textAlign':'center'}),
    html.P("Select year and atributes:"),
    dcc.Dropdown(
        id="dropdown21",
        options=atributos_num,
        value='population',
        clearable=False,
    ),dcc.Dropdown(
        id="dropdown22",
        options=atributos_num,
        value='electricity_demand',
        clearable=False,
    ),dcc.Dropdown(
        id="dropdown23",
        options=atributos_num,
        value='gdp',
        clearable=False,
    ),dcc.Dropdown(
        id="dropdown24",
        options=atributos_num,
        value='gdp per capita',
        clearable=False,
    ),dcc.Slider(df.year.min(), df.year.max(), step=1, value=2010, marks=None,id='my-slider2' ,
    tooltip={"placement": "bottom", "always_visible": True}),
    dcc.Graph(id="graph2")
])


@app2.callback(
    Output("graph2", "figure"), Input("my-slider2", "value"),
    Input("dropdown21", "value"),Input("dropdown22", "value"),
    Input("dropdown23", "value"),Input("dropdown24", "value"))

def dispersion(year,atributo_x="population", atributo_y="electricity_demand", atributo_tamaño="gdp",
               atributo_color="gdp per capita"):
    df_year=df[df.year==year]
    df_year=df_year[np.logical_not(np.isnan(df_year[atributo_tamaño]))]
    df_year=df_year[df_year.country!="World"]
    fig2 = px.scatter(df_year,x=atributo_x,y=atributo_y,color=atributo_color,size=atributo_tamaño,hover_data=["country"],
                     labels={
                     atributo_x: f"{atributo_x} ({dict_unidades[atributo_x]})",
                     atributo_y: f"{atributo_y} ({dict_unidades[atributo_y]})",
                     atributo_color: f"{atributo_color} ({dict_unidades[atributo_color]})",
                     atributo_tamaño: f"{atributo_tamaño} ({dict_unidades[atributo_tamaño]})"})
    fig2.update_layout(plot_bgcolor="white")
    fig2.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
    fig2.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
    return fig2


if __name__ == '__main__':
    webbrowser.open("http://127.0.0.1:7081/")
    app2.run(host='127.0.0.1', port='7081', proxy=None, debug=False,
             dev_tools_ui=None, dev_tools_props_check=None, dev_tools_serve_dev_bundles=None,
             dev_tools_hot_reload=None, dev_tools_hot_reload_interval=None,
             dev_tools_hot_reload_watch_interval=None,
             dev_tools_hot_reload_max_retry=None, dev_tools_silence_routes_logging=None,
             dev_tools_prune_errors=None)








## Visualización 3

In [7]:
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go

app3 = Dash(__name__)

app3.layout = html.Div([
    html.H1('World map',style={'textAlign':'center'}),
    html.P("Select year and atribute:"),
    dcc.Dropdown(
        id="dropdown31",
        options=atributos_num,
        value='gdp per capita',
        clearable=False,
    ),dcc.Slider(df.year.min(), df.year.max(), step=1, value=2010, marks=None,id='my-slider3' ,
    tooltip={"placement": "bottom", "always_visible": True}),
    dcc.Graph(id="graph3")
])


@app3.callback(
    Output("graph3", "figure"),
    Input("dropdown31", "value"),Input("my-slider3", "value"))

def worldmap(atributo,year):

    # generate values
    nueva_column=pd.Series(data=np.repeat(np.nan,world.shape[0]))
    for i in range(world.shape[0]):
        try: nueva_column.iloc[i]=df[(df.year==year)&(df.iso_code==world.iloc[i].iso_a3)][atributo].values[0]
        except: nueva_column.iloc[i]=np.nan

    world.pop_est=nueva_column

    # plot a basic map of the world
    fig3 = px.choropleth(world, geojson=world.geometry, locations=world.index,
                        color="pop_est", hover_data=["name"],
                       labels={
                     "pop_est": f"{atributo} ({dict_unidades[atributo]})"})
    fig3.update_geos(fitbounds="locations", visible=False)
    #fig3.update_layout(coloraxis_colorbar=dict(len=0.5, thickness=10))
    return fig3

if __name__ == '__main__':
    webbrowser.open("http://127.0.0.1:7082/")
    app3.run(host='127.0.0.1', port='7082', proxy=None,
                                    debug=False, dev_tools_ui=None, dev_tools_props_check=None,
                                    dev_tools_serve_dev_bundles=None, dev_tools_hot_reload=None,
                                    dev_tools_hot_reload_interval=None, dev_tools_hot_reload_watch_interval=None,
                                    dev_tools_hot_reload_max_retry=None, dev_tools_silence_routes_logging=None,
                                    dev_tools_prune_errors=None)





In [8]:
#!jupyter nbconvert --execute --to 'html' practica_parte2.ipynb

[2024-04-20 09:32:40,209] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\jglez\anaconda3\Lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jglez\anaconda3\Lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jglez\anaconda3\Lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jglez\anaconda3\Lib\site-packages\flask\app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jglez\anaconda3\Lib\site-packages\dash\dash.py", line 1352, in dispatch
    ctx.run(
  F